### FBP: Frequency Band Power

In [114]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from scipy.io import loadmat
from pathlib import Path
import matplotlib.patches as patches
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa
from scipy import stats
import mne
import eeg_analysis.funcs4eeg as fe
import re
import imp
import ast
import behavior.func4behav as fb
imp.reload(fe)
imp.reload(fb)

<module 'behavior.func4behav' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\attention_tES\\tes-attention\\processing\\behavior\\func4behav.py'>

In [115]:
real_ids = [1, 3, 4, 5, 9, 12, 13, 17, 18]
sham_ids = [2, 6, 7, 8, 10, 11, 14, 15, 16]

bands = [[4, 7], [8, 12], [12.5, 30], [30, 60], [60, 100]]
band_names = ['theta', 'alpha', 'beta', 'gamma', 'high gamma']

### power over the entire trial

In [116]:
case = 'endo'
# sham_before, sham_after, real_before, real_after = fe.pipeline_FBP_allsubs(case)

In [120]:
subject_id = 12
behav_sham_before, behav_sham_after, behav_real_before, behav_real_after, rt_means, rt_std_errors = fe.reaction_time_table(case)
trials_before, trials_after = fe.get_inuse_trials(subject_id, behav_real_before, behav_real_after)
eeg_before, eeg_after = fe.load_eeg(subject_id)
events, event_dict = fe.make_default_events(eeg_before)
picked_events, picked_events_dict = fe.make_custom_events(eeg_before, events, event_dict, trials_before, fe.translate_case(case))


In [134]:
trial_fixations = picked_events[picked_events[:, 2] == 1]
trial_stims = picked_events[picked_events[:, 2] == 11]
trial_fixations[:, 0]
trial_start_times = trial_fixations[:,0] # time steps of fixation onsets
trial_end_times = (trial_stims[:,0] + (1200*1.5)).astype(int) # time steps of stimulus onsets

data = eeg_before.get_data()[1:33,:] # 32 channels

for i, end_time in enumerate (trial_end_times):
    trial = data[:,trial_start_times[i]:end_time]
    print(trial.shape[1]/1200)

5.035833333333334
5.031666666666666
5.034166666666667
5.031666666666666
4.535833333333334
4.533333333333333
4.533333333333333
4.534166666666667
5.030833333333334
4.534166666666667
5.0325
4.533333333333333
4.534166666666667
5.034166666666667
5.0325
4.533333333333333
5.0325
4.533333333333333
5.034166666666667
4.533333333333333
4.533333333333333
5.033333333333333
5.031666666666666
5.0325
5.031666666666666
4.531666666666666
4.5325
5.034166666666667
5.033333333333333
5.033333333333333
5.034166666666667
4.534166666666667
4.5325
4.533333333333333
5.030833333333334
5.033333333333333
5.0325
4.531666666666666
5.033333333333333
4.533333333333333
5.0325


In [131]:
data[:,trial_start_times[0]:7243].shape

(32, 6043)

In [119]:
# set baseline

### Evoked power change